# Document Classification with UDOP
In this notebook, we will give a sequential example on data processing and model IO for a document classification example

In [1]:
import torch

from core.models import UdopUnimodelForConditionalGeneration, UdopConfig, \
    UdopTokenizer

# Change path to the model checkpoints
config = UdopConfig.from_pretrained("udop-unimodel-large-224")
tokenizer = UdopTokenizer.from_pretrained("udop-unimodel-large-224")
model = UdopUnimodelForConditionalGeneration.from_pretrained("udop-unimodel-large-224")

['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_44>', '<extra_id_45>', '<extra_id_46>', '<extra_id_47>', '<extra_id_48>', '<extra_id_49>', '<extra_id_50>', '<extra_id_51>', '<extra_id_52>', '<extra_id_53>', '<extra_id_54>', '<extra_id_55>', '<extra_id_56>', '<extra_id_57>', '<extra_id_58>', '<extr

================              Failed loaded mae              Ignore this if you are finetuning the model              ================              


Some weights of UdopUnimodelForConditionalGeneration were not initialized from the model checkpoint at udop-unimodel-large-224 and are newly initialized: ['special_vis_token']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
from PIL import Image
from core.common.utils import img_trans_torchvision, get_visual_bbox
import json
import numpy as np
import torch

# Load example document and process them
im = Image.open('examples/00070353.png').convert('RGB')
im = img_trans_torchvision(im).unsqueeze(0)
visual_seg_data = get_visual_bbox(im.shape[-1]).unsqueeze(0)
annot = json.load(open('examples/00070353.json'))

all_bboxes = []
all_text = []

# Process task prefix, here we use document classification
task_prefix = 'document classification.'
sub_tokens = tokenizer.tokenize(task_prefix)
for sub_token in sub_tokens:
    all_text.append(sub_token)
    all_bboxes.append([0, 0, 0, 0])
        
for item in annot['form']:
    bb = np.array(item['box'])/1000.0
    sub_tokens = tokenizer.tokenize(item['text'])
    for sub_token in sub_tokens:
        all_text.append(sub_token)
        all_bboxes.append(bb)

all_text_ids = tokenizer.convert_tokens_to_ids(all_text)
input_ids = torch.tensor(np.array(all_text_ids)).unsqueeze(0)
input_seg_data = torch.tensor(np.array(all_bboxes)).unsqueeze(0)

In [3]:
# Inference from the inputs
output_ids = model.generate(
            input_ids,
            seg_data=input_seg_data,
            image=im,
            visual_seg_data=visual_seg_data,
            use_cache=True,
            decoder_start_token_id=None,
            num_beams=1,
            max_length=10,
        )
output_text = tokenizer.decode(output_ids[0][1:-1])
output_text

/home/v-terrantang/anaconda3/envs/mmdif/lib/python3.8/site-packages/transformers/modeling_utils.py:812: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/home/v-terrantang/anaconda3/envs/mmdif/lib/python3.8/site-packages/transformers/modeling_utils.py:769: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


'form'